# Decision Trees


In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles

# pip install git+git://github.com/mgelbart/plot-classifier.git
from plot_classifier import plot_classifier

import warnings
warnings.filterwarnings('ignore')

## Decision Trees

![toy tree](../images/toytree.PNG)

### Some Terminology

![structure](../images/treestructure.PNG)

In [ ]:
def plot_tree(X, y, model, predict_proba = False):
    
    # Join data for plotting
    sample = (X.join(y))
    # Create a mesh for plotting
    step = (X.max() - X.min()) / 50
    x1, x2 = np.meshgrid(np.arange(sample.min()[0]-step[0], sample.max()[0]+step[0], step[0]),
                         np.arange(sample.min()[1]-step[1], sample.max()[1]+step[1], step[1]))

    # Store mesh in dataframe
    mesh_df = pd.DataFrame(np.c_[x1.ravel(), x2.ravel()], columns=['x1', 'x2'])

    # Mesh predictions
    if predict_proba:
        mesh_df['predictions'] = model.predict_proba(mesh_df[['x1', 'x2']])[:, 0]
        # Plot
        base_plot = alt.Chart(mesh_df).mark_rect(opacity=0.5).encode(
            x=alt.X('x1', bin=alt.Bin(step=step[0])),
            y=alt.Y('x2', bin=alt.Bin(step=step[1])),
            color=alt.Color('predictions', title='P(blue)', scale=alt.Scale(scheme='redblue'))
        ).properties(
            width=400,
            height=400
        )
        return alt.layer(base_plot).configure_axis(
            labelFontSize=20,
            titleFontSize=20
        ).configure_legend(
            titleFontSize=20,
            labelFontSize=20
        )
    else:
        mesh_df['predictions'] = model.predict(mesh_df[['x1', 'x2']])
        # Plot
        scat_plot = alt.Chart(sample).mark_circle(
            stroke='black',
            opacity=1,
            strokeWidth=1.5,
            size=100
        ).encode(
            x=alt.X(X.columns[0], axis=alt.Axis(labels=True, ticks=True, title=X.columns[0])),
            y=alt.Y(X.columns[1], axis=alt.Axis(labels=True, ticks=True, title=X.columns[1])),
            color=alt.Color(y.columns[0])
        )
        base_plot = alt.Chart(mesh_df).mark_rect(opacity=0.5).encode(
            x=alt.X('x1', bin=alt.Bin(step=step[0])),
            y=alt.Y('x2', bin=alt.Bin(step=step[1])),
            color=alt.Color('predictions', title='Legend')
        ).properties(
            width=400,
            height=400
        )
        return alt.layer(base_plot, scat_plot).configure_axis(
            labelFontSize=20,
            titleFontSize=20
        ).configure_legend(
            titleFontSize=20,
            labelFontSize=20
        )

## Let's see how decision trees work
- Get our data in
- Train / validation split
- Fit a **classification tree**

In [ ]:
df = pd.read_csv('../data/cities_USA.csv', index_col=0)
X = df.drop(columns=['vote'])
y = df[['vote']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
df.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier(max_depth=None)
model.fit(X_train, y_train)

## What did the classification tree do?
- Found a good way to split and repeat


Let's go ahead and visualize the decision boundaries. 

In [ ]:
plot_tree(X_train, y_train, model)

And we can also print out a tree to see how the decisions were made. We'll go ahead and save this to a file and just bring it in to visualize it a bit better.

In [ ]:
from sklearn.tree import export_graphviz 
# conda install python-graphviz

import graphviz

In [ ]:
dot_data = export_graphviz(model)
graphviz.Source(export_graphviz(model,
                                out_file=None,
                                feature_names=X.columns,
                                class_names=["red", "blue"],
                                impurity=True))

A look at our full tree

![full tree](../images/graphviztree.PNG)



and our decision boundaries

![decision boundaries](../images/decisionboundaries.gif)


## So how do we decide the split?
- Basic idea is to pick a criterion (see [here](https://scikit-learn.org/stable/modules/tree.html#mathematical-formulation)) and then minimize it across possible splits.
- Common one is Gini impurity

### Gini Impurity

First, Gini Impurity is the probability that a novel classification will be classified incorrectly. So, if we split the data off of some condition, what is the impurity of each of the new nodes? However, not all nodes are even, so can we weight our impurity of the new nodes to get at what is the best split to make?

<img src='../images/gini.png' width=300>

- $C$ is number of classes in target variable
- $p$ is proportion of class $i$ in a group
- tells us what is the probability of misclassifying an observation --> lower the better


Full details here: https://towardsdatascience.com/gini-impurity-measure-dbd3878ead33

![gini impurity](../images/giniimpurity.PNG)

## Let's try writing some code for Gini Impurity for a given node

The first thing we need to do is create a function to calculate the gini impurity of a node.

In [ ]:
vals = np.array(['yes', 'no', 'yes', 'no', 'yes', 'yes', 'no'])

def calculate_gini_impurity(vals):
    (unique_vals, counts) = np.unique(vals, return_counts=True)
    num_vals = len(vals)
    gini = 0
    for i in range(len(unique_vals)):
        num_class = counts[i]
        gini = gini + ((num_class/num_vals) * (1 - num_class/num_vals))
    return gini
        
        
print(calculate_gini_impurity(vals))
print(calculate_gini_impurity(np.array([1, 2])))
    

## Data 

path: `/src/data/does_james_eat.csv`
description: some observations as to whether or not James's consumes food.

Based off of whether or not James is hungry and whether or not they have Pop Tarts, can we predict whether James will eat?

How should we split the first node?

In [ ]:
eat = pd.read_csv("../data/does_james_eat.csv")
eat

But first, what if we were just to throw out a guess on whether or not James eat's food without any inputs? Well, that's simply calculating the impurity of the target `will_eat` for the entire dataset.

In [ ]:
calculate_gini_impurity(eat['will_eat'])

Let us now split our data off of some condition. How about we split on the variable `is_hungry`

In [ ]:
target = 'will_eat'
split = eat['is_hungry'] == 'yes';
hungry = eat[split]
not_hungry = eat[~split]

print("hungry == yes")
print(hungry)
print("hungry == no")
print(not_hungry)

In [ ]:
print(calculate_gini_impurity(hungry['will_eat']))
print(calculate_gini_impurity(not_hungry['will_eat']))

Now, let's caculate the impurity of the split of the sum of the weighted impurities for the new nodes. Whatever split minimizes this value is where we split this node.

In [ ]:
def calculate_impurity(data, target_var, split_var, split_condition):
    num_samples = data.shape[0]
    split = data[split_var] == split_condition
    split_data0 = data[split]
    split_data1 = data[~split]
    weight0 = split_data0.shape[0] / num_samples
    weight1 = split_data1.shape[0] / num_samples
    gini0 = calculate_gini_impurity(split_data0[target_var])
    gini1 = calculate_gini_impurity(split_data1[target_var])
    return weight0 * gini0 + weight1 * gini1

print("gini impurity for has poptarts")
print(calculate_impurity(eat, 'will_eat', 'has_poptarts', 'yes'))

print("gini impurity for is hungry")
print(calculate_impurity(eat, 'will_eat', 'is_hungry', 'yes'))



## Regression trees
Imagine that our output (target) variable was continuous instead of discrete. In other words, we have a regression problem instead of a classification problem. Well, trees are an effective method in these scenarios as well.

A couple of things:
- Less intuitive, but same idea
- Instead of Gini, we use something that works for regression, like MSE (think residuals)

### Example

In [ ]:
from sklearn.tree import DecisionTreeRegressor

For the sake of example, we're going to go ahead and create some dummy data just for demonstration purposes. Here, we'll have two features and a target that we are trying to predict

In [ ]:
np.random.seed(333)
df = pd.DataFrame({'feature_1': np.random.randint(0, 10, (6,)),
                   'feature_2': np.random.randint(0, 10, (6,)),
                   'target': np.random.randint(0, 10, (6,))})
df

In [ ]:
X = df.drop(columns='target')
y = df['target']
model = DecisionTreeRegressor(max_depth=3).fit(X, y)
dot_data = export_graphviz(model)
graphviz.Source(export_graphviz(model,
                                out_file=None,
                                feature_names=X.columns))

## Visualizing the First cut - regression tree


![regression tree](../images/graphviztree2.PNG)

In [ ]:
scatter = alt.Chart(df).mark_circle(
    size=150,
    opacity=1,
    stroke='black',
    strokeWidth=1
).encode(
    x=alt.X('feature_1', title='feature_1'),
    y=alt.Y('feature_2', title='feature_2'),
    color='target')
text = scatter.mark_text(
    align='left',
    baseline='middle',
    fontSize = 20,
    dx=8
).encode(
    text='target')
split = alt.Chart(df).mark_rule().encode(
    x='a:Q',
    size=alt.SizeValue(2),
    color=alt.ColorValue("red")
).transform_calculate(
    a="6.5"
)
alt.layer(scatter, text, split).configure_axis(
    labelFontSize=20,
    titleFontSize=20
).configure_legend(
    titleFontSize=20,
    labelFontSize=20
).properties(
    width=300,
    height=300
)

## Decision trees, pros and cons
### Advantages:
- Simple to understand and to interpret. Trees can be visualised.
- Requires little data preparation. Doesn't require data normalisation.
- Can handle multi-class classification well.

### Disadvantages:
- Tendency to overfit with overly complex trees that don't generalize well; pruning techniques (e.g., minimum number of samples required at a leaf node, maximum depth of tree) is needed to avoid the problem
- Decision trees can be unstable because small variations in the data might result in a completely different tree being generated; mitigated through ensembles (more on this soon)
- They suffer from an inherent instability, since due to their hierarchical nature, the effect of an error in the top splits propagate down to all of the splits below.
- Is a "greedy" algorithm. Each node is only locally optimal which cannot gaurantee globally optimal tree

## Random Forests

![random forests](../images/randomforest1.PNG)

## Imagine the following scenario

You want to be able to be able to classify someone's fast food preference based off some questions. Perhaps, you ask questions such as "are you a vegetarian/vegan?", "do you like hamburgers?", "do you like spicy food?" And perhaps the range of target responses are "McDonalds", "Chipotle" and "KFC". 

- A *decision tree* would be like going out to Times Square and asking everyone **all** of those questions and getting their favorite fast food restaurant as a response. Then we can build a model off of all of their responses.

- A *random forest* is like having a set of interviewers and each interviewer only selects a subset of the questions to ask the people they go up to. From this each of these interviewers, we build a tree and when we get new inputs, we ask each interviewer what their tree would predict and we take the average response as our response. 
    - note: a single individual can be sampled by many interviewers.
    
    

### General idea:
   - `fit` a diverse set of classifiers by **injecting randomness** in the classifier construction
   - `predict` by taking the average of predictions given by individual classifiers

### How do we inject randomness in the classifier construction? 
   1. Data: Build each tree on a bootstrap sample (i.e., a sample drawn **with replacement** from the training set)
   2. Features: Consider a random subset of features at each split (`RandomForestClassifier`)
        
**The intuition here is the wisdom of crowds**

### Bootstrapping samples

<img src='../images/bootstrapping.PNG' width='300'>

## The random forest classifier 

- Create a collection (ensemble) of trees. Grow each tree on an independent bootstrap sample from the data.
- At each node:
    - Randomly select a subset of features out of all features (independently for each node)
    - Find the best split on the selected features
    - Grow the trees to maximum depth
- Vote between the trees to get predictions for new data

## Random Forest Training
- Showing feature subset

<img src='../images/random-forest-features.jpg'>

## Random Forest Prediction
<img src='../images/random-forest-predict.jpg'>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def make_num_tree_plot(X_train, y_train, X_test, y_test, num_trees):
    """
    Make number of trees vs error rate plot for RandomForestClassifier
   
    Parameters
    ----------
    model: sklearn classifier model
        The sklearn model
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_test: numpy.ndarray        
        The X part of the test/validation set
    y_test: numpy.ndarray
        The y part of the test/validation set    
    num_trees: int
        The value for `n_estimators` argument of RandomForestClassifier
    Returns
    -------
        None
        Shows the number of trees vs error rate plot
            
    """    
    train_err = []
    test_err = []
    for ntree in num_trees:
        model = RandomForestClassifier(n_estimators=ntree)
        model.fit(X_train, y_train)
        train_err.append(1-model.score(X_train, y_train))
        test_err.append(1-model.score(X_test, y_test))

    plt.semilogx(num_trees,train_err,label="train");
    plt.semilogx(num_trees,test_err,label="test");
    plt.legend();
    plt.xlabel('number of trees');
    plt.ylabel('error rate');

## Beating the fundamental tradeoff...

- Decreasing training error and not increasing test error.
- This is the promise of ensembles, though it's not guaranteed

In [ ]:
make_num_tree_plot(X_train, np.ravel(y_train), X_val, np.ravel(y_val), (1,5,10,25,50,100,200,500))

## Random Forests summary

- Accuracy
    - Usually more accurate compared to decision trees, usually one of the best performing off-the-shelf classifiers
- Speed
    - Slower than decision trees because we are fitting multiple trees 
    - But can easily parallelize training because all trees are independent of each other - take seperate data and features 
- Overfitting
    - No depth limit decision tree tends to overfit
    - Random forests are less likely to overfit
- Interpretability
    - Decision trees are more interpretable

## Ensembling - why?
- Weak learners (or base models) are models that can be used as building blocks for designing more complex models
- By combining several weak learners together, we can try to reduce bias and/or variance of weak learners
- The ensemble model could have better performance - Kaggle competition winning models are usually ensemble methods

### Some disadvantages:
- Ensembling increases computational time to fit and predict
- Also decreases interpretability

## More Ensemble Methods

### Bagging (Bootstrap and Aggregate)
<img src='../images/bagging.PNG' width='700'>

- Random forest is an example of this
- Same type of weak model used (e.g., trees), models learned in parallel, combines them in a deterministic process (e.g., averaging)


### Boosting (will show example)
<img src='../images/boosting.PNG' width='700'>

- Add one "learner" (i.e. model) at a time, which addresses the "shortcomings" of the current ensemble.
- Unlike averaging, ensemble is created *_during training_*, not after. The process is sequential.
- Example: `AdaBoost`, `XGBoost`, `LGBM`


### Stacking (will show example)
<img src='../images/stacking.PNG' width='700'>

- Usually with heterogeneous (different base model types) models
- Each model learns indepedently and is combined using a voting model


src = https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205

## Load smaller version of data

In [ ]:
with open("../data/citiesSmall.pkl", "rb") as f:
    data = pickle.load(f)
    X_train = data['X']
    y_train = data['y']

    X_val = data['Xtest']
    y_val = data['ytest']

## Boosting - one example library
- `AdaBoost`: Adaptive Boosting

<img src='../images/adaboost.PNG'>

## Demo

In [ ]:
# Let's generate some fake data

# generate blobs with fixed random generator
n = 100
n_classes = 2
X, y = make_circles(n_samples=n, random_state = 122)

X_train_circ, X_test_circ, y_train_circ, y_test_circ = train_test_split(X, y, test_size=0.2)

# How does the data look like? 
plt.scatter(*X_train_circ.T, c=y_train_circ, marker='.', cmap='Dark2');

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ensemble = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators = 4)
ensemble.fit(X_train_circ, y_train_circ);

In [ ]:
# Combine weak learners 
plt.figure(figsize=(20,20))
count = 0
weights = ensemble.estimator_weights_
for estimator in ensemble.estimators_: # needs scikit-learn-0.20
    plt.subplot(4,2,count+1)
    plot_classifier(X_train_circ, y_train_circ, estimator, ax=plt.gca());
    tr_err = (1 - estimator.score(X_train_circ, y_train_circ))
    te_err = (1 - estimator.score(X_test_circ, y_test_circ))
    title = 'Round: %i; Train_error: %0.3f; Test error: %0.3f; estimator weight: %0.3f'%(count+1, tr_err, te_err, weights[count])
    plt.title(title);
    count += 1
    
tr_err = (1 - ensemble.score(X_train_circ, y_train_circ))
te_err = (1 - ensemble.score(X_test_circ, y_test_circ))
title = 'Ensemble model; Train_error: %0.3f; Test error: %0.3f'%(tr_err, te_err)
plt.subplot(4,2,count)
plt.title(title)
plot_classifier(X_train_circ, y_train_circ, ensemble, ax=plt.gca());        

## Stacking - code

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier  #meta model

In [ ]:
# Create a simple voting classifier using scikit-learn's VotingClassifier

classifiers = {
    "Decision tree"         : DecisionTreeClassifier(max_depth=5),
    "KNN"                   : KNeighborsClassifier(),
    "Naive Bayes"           : GaussianNB(),
    "Logistic Regression"   : LogisticRegression(),
}


ensemble = VotingClassifier(classifiers.items(), voting="soft")

# note: sklearn doesn't seem to support passing pre-trained classifiers into VotingClassifier
ensemble.fit(X_train, np.ravel(y_train));

print('Ensemble performance: \n')
print("Training error:   %.2f" % (1-ensemble.score(X_train, np.ravel(y_train))))
print("Validation error: %.2f" % (1-ensemble.score(X_val, np.ravel(y_val))))

In [ ]:
## What's the performance of individual classifiers in the ensemble?
for name, estimator in ensemble.named_estimators_.items(): # needs scikit-learn-0.20
    tr_err = (1 - estimator.score(X_train, np.ravel(y_train)))
    te_err = (1 - estimator.score(X_val, np.ravel(y_val)))
    print('%s: Train_error: %0.3f; Test error: %0.3f'%(name, tr_err, te_err))

In [ ]:
plt.figure(figsize=(16,16))
count = 1
for name, estimator in ensemble.named_estimators_.items(): # needs scikit-learn-0.20
    plt.subplot(3,2,count)
    plot_classifier(X_train, y_train, estimator, ax=plt.gca());
    tr_err = (1 - estimator.score(X_train, y_train))
    te_err = (1 - estimator.score(X_val, y_val))
    title = '%s: Train_error: %0.3f; Test error: %0.3f'%(name, tr_err, te_err)    
    plt.title(title);
    count += 1
    
plt.subplot(3,2,6)
plot_classifier(X_train, y_train, ensemble, ax=plt.gca());
tr_err = 1 - ensemble.score(X_train, y_train)
te_err = 1 - ensemble.score(X_val, y_val)
title = '%s; Train_error: %0.3f; Test error: %0.3f'%('Ensemble', tr_err, te_err)    
plt.title(title);